###1) Download Libraries

In [0]:
%pip install databricks-sdk --upgrade databricks-vectorsearch langchain==0.2.0 langchain_community transformers databricks-cli mlflow #torch TensorFlow 

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.1/497.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 59.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━

In [0]:
dbutils.library.restartPython()

###2) Load dataset, configure secrets & set up


In [0]:
#Fill this out with the catalog where the data is stored and where all objects will be created (eg: tables, models etc)
catalog = "catalog_name" 

#Fill this out with the schema where the data is stored and where all objects will be created (eg: tables, models etc)
schema = "schema_name" 

In [0]:
spark.sql(f"""USE CATALOG {catalog}""")
spark.sql(f"""USE DATABASE {schema}""")

DataFrame[]

In [0]:
#Name of table to be replaced if different
workorder = spark.read.table("engineering_notes") 

In [0]:
workorder.display()

id,investigation_notes,resolution_notes
1,Bearings of the pump indicate wear and tear that show signs of damage,"Simon harris will first try to lubricate and align the bearing to see if it would resolve the issue. If that does not work, he will replace the bearings next Friday (23/10/24)"
2,Pump seal does not seem to be working upon inspection,Check seal installation and ensure that it's compatible to the pumped fluid. The seal chamber pressure and temperature also needs to be checked
3,"Pump showed signs of wear and tear, there seems to be corrosion and erosion after 15 years, which is causing weakening of the pump body as well as contamination to the fluids flowing through as debris was found",James had applied special coating but would need to replace the pump with materials resistant to corrosion such as stainless steel if it does not resolve the issue
4,Leakage seems to appear around pump seals,Paul Connie inspected and reinstalled the seal. If it continues the seal would need to be replaced
5,Corrosion around the pump impellers which is causing reduced performance seen with low pressure lately,Arun will be checking the material and applying coating to see if it would help. Another follow up action is putting in a schedule for maintenance for week commencing 03/11/23
6,Pump is creating lots of noise and has reduced flow rate,Checked the pump and Jonny ordered a replacement pump as there seems to be mechanical damage to the pump. The part is expected to arrive next week on 04/06/23
7,"Pump has stopped completely, this is critical and require immediate resolution. This has been escalated to Jonny whos the engineer on duty and he will have an inspection first thing in the morning.",Tried restarting the pump but not working. New pump taken from store and installed on site within a few hours. Replacement ordered for the back up pump which will arrive in a few weeks
8,Pump hoses have come off,Inspected and reattached the hoses. Ran the pump for a few hours after and everything is working fine
9,New pump installed last week is leaking and cause loud noises. Pump pressure so far seems to be unaffected,Checked and there was a misalignment with the pump internal fixings. This has been fixed. A regular maintenance needs to be done to ensure that it doesn't go out of alignment
10,Bearings of the pump are worn out,Replaced bearings and ordered replacement for back up. Pump has been working fine since


In [0]:
#Replace the names below with the names of the endpoints for embedding model & chat model that would have been pre provisioned within the workspace. Replace the name for vector search endpoint & the name of the table to write to.
embedding_endpoint_name = "bge_m3"
chat_endpoint_name = "llama_instruct"
vector_search_endpoint_name = "shared-endpoint"
table_name = "table_name_embeddings"

###3) Chunk the contents 



### Splitting documentation pages into small chunks

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-prep-2.png?raw=true" style="float: right; width: 600px; margin-left: 10px">

LLM models typically have a maximum input context length, and you won't be able to compute embeddings for very long texts.
In addition, the longer your context length is, the longer it will take for the model to provide a response.

Document preparation is key for your model to perform well, and multiple strategies exist depending on your dataset:

- Split document into small chunks
- Truncate documents to a fixed length
- The chunk size depends on your content and how you'll be using it to craft your prompt. Adding multiple small doc chunks in your prompt might give different results than sending only a big one
- Split into big chunks and ask a model to summarize each chunk as a one-off job, for faster live inference
- Create multiple agents to evaluate each bigger document in parallel, and ask a final agent to craft your answer...


In this demo, we have big documentation articles, which are too long for the prompt to our model. We won't be able to use multiple documents as RAG context as they would exceed our max input size. Some recent studies also suggest that bigger window size isn't always better, as the LLMs seem to focus on the beginning and end of your prompt.

In our case, we'll split these articles to ensure that each chunk is less than 500 tokens using LangChain. 

#### LLM Window size and Tokenizer

The same sentence might return different tokens for different models. LLMs are shipped with a `Tokenizer` that you can use to count tokens for a given sentence (usually more than the number of words) (see [Hugging Face documentation](https://huggingface.co/docs/transformers/main/tokenizer_summary) or [OpenAI](https://github.com/openai/tiktoken))

Make sure the tokenizer you'll be using here matches your model. We'll be using the `transformers` library to count llama2 tokens with its tokenizer. This will also keep our document token size below our embedding max size (1024).

<br/>
<br style="clear: both">
<div style="background-color: #def2ff; padding: 15px;  border-radius: 30px; ">
  <strong>Information</strong><br/>
  Remember that the following steps are specific to your dataset. This is a critical part to building a successful RAG assistant.
  <br/> Always take time to manually review the chunks created and ensure that they make sense and contain relevant information.


In [0]:
from pyspark.sql.functions import pandas_udf
import pandas as pd
from transformers import  AutoTokenizer
from langchain.text_splitter  import RecursiveCharacterTextSplitter 
from pyspark.sql import types as T


tokenizer = AutoTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=500, chunk_overlap=50)

def get_chunks(c):
  chunks = []

  #Check if the chunks are small, if so add to them until they reach the midpoint of the max token size
  if len(tokenizer.encode(c)) <= 500/2:
    chunks.append(c)
  
  elif len(tokenizer.encode(c)) > 500/2:
    chunks.extend(text_splitter.split_text(c.strip()))

  return chunks


@pandas_udf(T.ArrayType(T.StringType()))
def get_chunks_udf(result: pd.Series) -> pd.Series:
  return result.apply(get_chunks)

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Unexpected internal error when monkey patching `PreTrainedModel.from_pretrained`: 
PreTrainedModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.

Unexpected internal error when monkey patching `Trainer.train`: 
Trainer requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.



tokenizer_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [0]:
from pyspark.sql import functions as F
workoder_chunked = workorder.withColumn("chunks", get_chunks_udf("investigation_notes")) \
                   .withColumn("chunk_exploded",F.explode("chunks")) \
                   .dropna(how = "any", subset = ['chunk_exploded']) \
                   .drop("chunks")

In [0]:
workoder_chunked.display()

id,investigation_notes,resolution_notes,chunk_exploded
1,Bearings of the pump indicate wear and tear that show signs of damage,"Simon harris will first try to lubricate and align the bearing to see if it would resolve the issue. If that does not work, he will replace the bearings next Friday (23/10/24)",Bearings of the pump indicate wear and tear that show signs of damage
2,Pump seal does not seem to be working upon inspection,Check seal installation and ensure that it's compatible to the pumped fluid. The seal chamber pressure and temperature also needs to be checked,Pump seal does not seem to be working upon inspection
3,"Pump showed signs of wear and tear, there seems to be corrosion and erosion after 15 years, which is causing weakening of the pump body as well as contamination to the fluids flowing through as debris was found",James had applied special coating but would need to replace the pump with materials resistant to corrosion such as stainless steel if it does not resolve the issue,"Pump showed signs of wear and tear, there seems to be corrosion and erosion after 15 years, which is causing weakening of the pump body as well as contamination to the fluids flowing through as debris was found"
4,Leakage seems to appear around pump seals,Paul Connie inspected and reinstalled the seal. If it continues the seal would need to be replaced,Leakage seems to appear around pump seals
5,Corrosion around the pump impellers which is causing reduced performance seen with low pressure lately,Arun will be checking the material and applying coating to see if it would help. Another follow up action is putting in a schedule for maintenance for week commencing 03/11/23,Corrosion around the pump impellers which is causing reduced performance seen with low pressure lately
6,Pump is creating lots of noise and has reduced flow rate,Checked the pump and Jonny ordered a replacement pump as there seems to be mechanical damage to the pump. The part is expected to arrive next week on 04/06/23,Pump is creating lots of noise and has reduced flow rate
7,"Pump has stopped completely, this is critical and require immediate resolution. This has been escalated to Jonny whos the engineer on duty and he will have an inspection first thing in the morning.",Tried restarting the pump but not working. New pump taken from store and installed on site within a few hours. Replacement ordered for the back up pump which will arrive in a few weeks,"Pump has stopped completely, this is critical and require immediate resolution. This has been escalated to Jonny whos the engineer on duty and he will have an inspection first thing in the morning."
8,Pump hoses have come off,Inspected and reattached the hoses. Ran the pump for a few hours after and everything is working fine,Pump hoses have come off
9,New pump installed last week is leaking and cause loud noises. Pump pressure so far seems to be unaffected,Checked and there was a misalignment with the pump internal fixings. This has been fixed. A regular maintenance needs to be done to ensure that it doesn't go out of alignment,New pump installed last week is leaking and cause loud noises. Pump pressure so far seems to be unaffected
10,Bearings of the pump are worn out,Replaced bearings and ordered replacement for back up. Pump has been working fine since,Bearings of the pump are worn out


### 4) Create embeddings & set up vector search

Databricks provides multiple types of vector search indexes:

- **Managed embeddings**: you provide a text column and endpoint name and Databricks synchronizes the index with your Delta table 
- **Self Managed embeddings**: you compute the embeddings and save them as a field of your Delta Table, Databricks will then synchronize the index
- **Direct index**: when you want to use and update the index without having a Delta Table

In this demo, we will show you how to setup a **Self-managed Embeddings** index. 

To do so, we will have to first compute the embeddings of our chunks and save them as a Delta Lake table field as `array&ltfloat&gt`

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/databricks-vector-search-type.png?raw=true" style="float: right" width="800px">


### 4.1) Self managed embeddings - Provisioned Throughput

In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

def compute_embeddings(batch):
  response = deploy_client.predict(endpoint=embedding_endpoint_name, inputs={"input": batch})
  return response['data'][0]['embedding']

In [0]:
def apply_embedding(workorder_list):
  embedding_list = []
  for row in workorder_list:
    embedding_dict = {}
    embedding_dict['id'] = row['id']
    embedding_dict['investigation_notes'] = row['investigation_notes']
    embedding_dict['resolution_notes'] = row['resolution_notes']
    embedding_dict['chunk_exploded'] = row['chunk_exploded']
    embedding_dict['embeddings'] = compute_embeddings(row['chunk_exploded'])
    embedding_list.append(embedding_dict)
  
  return embedding_list

In [0]:
workorder_to_embed_list = [row.asDict() for row in workoder_chunked.collect()]

In [0]:
embeddings = spark.createDataFrame(apply_embedding(workorder_to_embed_list))

In [0]:
embeddings.display()

chunk_exploded embeddings id investigation_notes resolution_notes Bearings of the pump indicate wear and tear that show signs of damage List(-0.032249484211206436, 0.025158008560538292, -0.08146744221448898, 0.0021585654467344284, 0.007214914541691542, -0.05114572495222092, 0.02747541479766369, 0.07017949223518372, -0.03647904470562935, 0.026195554062724113, 6.968543748371303E-4, 0.024158131331205368, 0.016078252345323563, 0.03215371444821358, 0.02618994377553463, -0.017579687759280205, 0.0229820366948843, 0.01811249367892742, 0.008350617252290249, 0.020709464326500893, -0.0422004833817482, 0.02981565147638321, -0.009333474561572075, -0.014669813215732574, -0.02085275761783123, 0.015548945404589176, -0.06349489092826843, -0.02673531137406826, 0.017853975296020508, 0.025995105504989624, -0.014944475144147873, -0.03620142117142677, 0.011286212131381035, 0.025615956634283066, -0.012228384613990784, -0.045976392924785614, 0.0035803753416985273, 0.018505968153476715, -0.03598235175013542, 0.04603567346930504, -0.00463491166010499, -0.028469275683164597, 0.0014392767334356904, -0.03690669685602188, 0.03856320679187775, -0.016978293657302856, -0.043456368148326874, -0.014537208713591099, -0.017423458397388458, -0.04099931940436363, 9.067435166798532E-4, -0.01843401789665222, 0.029810044914484024, -0.010927855037152767, -0.012942223809659481, 0.008807708509266376, -0.02328868955373764, 0.006653838325291872, -0.08253806829452515, -0.01916022039949894, -0.04245028644800186, 0.013977263122797012, -0.04843851178884506, -0.01352681778371334, -0.02647407352924347, 0.07287391275167465, -0.005501274019479752, 0.002546382136642933, -0.048562776297330856, -0.060901716351509094, 0.025601064786314964, 0.019826166331768036, -0.0010710302740335464, -0.030696142464876175, -0.07294970005750656, 0.022088564932346344, -0.010199499316513538, -0.055796533823013306, 0.0051423367112874985, 0.008389740251004696, 0.002484340453520417, 0.04059867188334465, 0.017090758308768272, 0.015470213256776333, -0.02591383457183838, 0.036950960755348206, -0.03742637485265732, -0.07472442090511322, -0.05247317999601364, 0.0026421521324664354, 0.027328649535775185, 0.01833379827439785, 0.00213632732629776, 0.008594052866101265, -0.03777645155787468, 0.010559508576989174, -0.06556971371173859, -0.03929271921515465, 0.01555697899311781, -0.03261147812008858, 0.003976717125624418, -0.020084286108613014, -0.01976579800248146, -0.05826370045542717, 0.04429159685969353, -0.05046605318784714, 0.0442991629242897, 0.025688478723168373, 0.011326280422508717, -0.014997665770351887, 0.010578776709735394, 0.0281804408878088, 0.025784289464354515, 0.017407609149813652, 0.005731008481234312, -0.006712779402732849, -0.008954277262091637, -0.026095978915691376, 0.015126686543226242, 0.005899612791836262, 0.009916580282151699, 0.051200103014707565, 0.015283768065273762, -0.054664984345436096, -0.04871464893221855, -0.030841445550322533, 0.02008933015167713, -0.007681734394282103, 0.06138860434293747, -0.004254903178662062, -0.01155187375843525, 0.01240493729710579, -0.05344183370471001, 0.017247026786208153, -0.04606251046061516, -0.015154419466853142, 0.01929572969675064, 0.008410785347223282, -0.02183058112859726, -0.021053360775113106, -0.0052283452823758125, 0.04423673450946808, -0.026393814012408257, -0.022172419354319572, 0.03897388279438019, -0.013991769403219223, 0.00635420810431242, -0.023247409611940384, 0.008184311911463737, -0.001373753766529262, -0.013495727442204952, -0.015402913093566895, 0.02518806979060173, 0.0068002077750861645, -0.04146665707230568, 0.04023048281669617, 0.015222693793475628, 0.010768646374344826, -0.031446970999240875, -0.005249564070254564, 0.03223040699958801, -0.024445790797472, 0.034090954810380936, 0.033043023198843, 0.023074178025126457, -0.008523075841367245, 0.05962700396776199, 0.021524393931031227, 0.010372974909842014, 0.007643516641110182, 0.025634294375777245, -0.00645326217636466, 0.001692323014140129, 0.004639885388314724, -0.00793646927922

In [0]:
#embedding_df.write.mode("overwrite").saveAsTable("workorders_embedding_manual")a

In [0]:
embeddings.write.option("mergeSchema",True).mode("overwrite").saveAsTable(table_name)

In [0]:
#This only needs to be configured once
spark.sql(f"""ALTER TABLE {catalog}.{schema}.{table_name} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)""")

DataFrame[]

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

vs_index_fullname = f"{catalog}.{schema}.engineering_notes_pt_self_managed_embeddings"

def create_or_update_index():
  if vs_index_fullname not in [i['name'] for i in vsc.list_indexes(vector_search_endpoint_name)['vector_indexes']]:
    vsc.create_delta_sync_index(
      endpoint_name=vector_search_endpoint_name,
      index_name=vs_index_fullname,
      source_table_name=f"{catalog}.{schema}.{table_name}",
      primary_key="id",
      pipeline_type="TRIGGERED",
      embedding_dimension=1024,
      embedding_vector_column="embeddings"
    )
    print("Creating_index")
  else: 
    vsc.get_index(vector_search_endpoint_name, vs_index_fullname).sync()
    print("Updating_index")

create_or_update_index()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
Updating_index


###4.2) Databricks managed embeddings - Provisioned Throughput

In [0]:

spark.sql(f"""ALTER TABLE {catalog}.{schema}.{table_name} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)""")

DataFrame[]

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

vs_index_fullname = f"{catalog}.{schema}.{table_name}_db_managed_embeddings"

def create_or_update_index():
  if vs_index_fullname not in [i['name'] for i in vsc.list_indexes(vector_search_endpoint_name)['vector_indexes']]:
    vsc.create_delta_sync_index(
      endpoint_name=vector_search_endpoint_name,
      index_name=vs_index_fullname,
      source_table_name=f"{catalog}.{schema}.{table_name}",
      primary_key="id",
      pipeline_type="TRIGGERED",
      embedding_source_column = "chunks",
      embedding_model_endpoint_name = embedding_endpoint_name
    )
    print("successfully created")

  else: 
    vsc.get_index(vector_search_endpoint_name, vs_index_fullname).sync()
    print("Index updated")

create_or_update_index()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
Index updated


###5) Query vector search endpoint 
* Query self managed embeddings
* Query Databricks managed embeddings

In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

#Note that we're querying a self managed embedding, so we need to convert to embeddings using our model endpoint
vs_index_fullname = f"{catalog}.{schema}.{table_name}_self_managed_embeddings"
query = "Issues related to pump failure?"

#Get the embeddings that are most similar to the query
def search_similar_content(query):
  results = vsc.get_index(vector_search_endpoint_name, vs_index_fullname).similarity_search(
    #Compute the embeddings using our model endpoint through query_vector
    query_vector=compute_embeddings(query),
    columns=["id", "investigation_notes","resolution_notes"],
    num_results=5)

  return [i[1] for i in results['result']['data_array']]

results_array = search_similar_content(query)
print(results_array)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
['CHECK FOUL PUMP NO 2 FOR SEAL LEAK FAILURE', 'REPAIR TO SUMP PUMP HOSES HAVE CAME OFF', 'CRITICAL - POWER GETTING TO STATION BUT NOTHING WORKING - NOPOWER AT ALL - NOT A HYDRO PROBLEM', 'CRITICAL: REPAIR / REPLACE PUMPS CRITICAL - PUMP NO.1 TRIPPING OUT AND BLOWING THE MAINSWITCH - OTHER PUMP NOT WORKING 09/07/2014 - HH - PUMPS ARRIVED YESTERDAY, JOB TO BEREPLANNED', 'CRITICAL--NO 2 PUMP IS CHOKED']


In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

response = deploy_client.predict(endpoint=chat_endpoint_name
                                 , inputs={"messages": [{"role":"user","content":f"Summarise the list of issues based on {query} in 100 words or less: {results_array}"}]})
print(response['choices'][0]['message']['content'])

Here is a summary of the issues related to pump failure in 100 words or less:

The list highlights several pump-related issues, including seal leak failures, faulty sump pump hoses, and power outages affecting pump operation. Pump No. 1 is tripping out and blowing the main switch, and Pump No. 2 is choked or experiencing a seal leak. Additionally, repairs or replacements are needed for critical pumps, with a plan needing to be reworked for the repair job.


In [0]:
from databricks.vector_search.client import VectorSearchClient
vsc = VectorSearchClient()

#Note that we're querying a DB managed embedding, where databricks will manage the creation of embeddings from the question
vs_index_fullname = f"{catalog}.{schema}.{table_name}_db_managed_embeddings"
query = "Issues related to pump failure?"

#Get the embeddings that are most similar to the query
def search_similar_content(query):
  results = vsc.get_index(vector_search_endpoint_name, vs_index_fullname).similarity_search(
    query_text=query,
    columns=["id", "investigation_notes","resolution_notes"],
    num_results=5)

  return [i[1] for i in results['result']['data_array']]

results_array = search_similar_content(query)
print(results_array)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().
['REPAIR TO SUMP PUMP HOSES HAVE CAME OFF', 'CRITICAL: REPAIR / REPLACE PUMPS CRITICAL - PUMP NO.1 TRIPPING OUT AND BLOWING THE MAINSWITCH - OTHER PUMP NOT WORKING 09/07/2014 - HH - PUMPS ARRIVED YESTERDAY, JOB TO BEREPLANNED', 'INLET FLOW METER FAULTY', 'CRITICAL - REMOVE CABLE FROM JUNCTION BOX FOR SUPERNATANTPUMP - WELL WILL FILL BACK UP AGAIN JON MORRIS IF POSS E004 - LF - 15/04/2014 - Quote from LGM 15/04/2014 - LF - PO 5/51155WGM Engineering del to ROGEN Uddintons Due 27/05 15/05/2014 - Paul Jones - quote from WGM for a pump repair - PENDING APPROVAL', 'WATER PIPE LEAK - SENTRIFUGE BUILDING - PLASTIC GLUED JOINTS']


In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")


response = deploy_client.predict(endpoint=chat_endpoint_name, inputs={"messages": [{"role":"user","content":f"Summarise the list of resolution for an engineering issue of {query} in 100 words or less: '{results_array}"}]})
print(response['choices'][0]['message']['content'])

Here is a summary of the issues related to pump failure in 100 words or less:

The list includes seven engineering issues related to pump failure. Five of the issues are critical, requiring immediate attention, including a faulty pump (Pump No.1), a non-functioning pump, and a faulty inlet flow meter. Two non-critical issues are a sump pump hose that came off and a water pipe leak. The issues also involve quotes and approvals for repairs, with some actions belonging to a specific person or date.


###6) Build langchain retriever
Use langchain wrapper to stitch together the steps required:
* Create embeddings for question
* Call the vector search index & find similar indexes
* Calling a chat model and pass in the response into a prompt template

Databricks Langchain wrapper makes it easy to do in one step, handling all the underlying logic and API call for you.

In [0]:
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings
from databricks.vector_search.client import VectorSearchClient
from langchain.llms import Databricks

vs_index_fullname = f"{catalog}.{schema}.{table_name}_self_managed_embeddings"
embedding_model = DatabricksEmbeddings(target_uri = "databricks",endpoint = embedding_endpoint_name)

#Wrap the retrieval of vectors in a retriever
def get_retriever(persist_dir: str = None):
  os.environ["DATABRICKS_HOST"] = host
  vsc = VectorSearchClient()
  vsc_index = vsc.get_index(
    endpoint_name = vector_search_endpoint_name, 
    index_name = vs_index_fullname
  )

  vectorstore = DatabricksVectorSearch(
    vsc_index, text_column = "investigation_notes", embedding = embedding_model
  )
  return vectorstore.as_retriever()

vectorstore = get_retriever()

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
#This is to format the input to the model
def reformat_Func(prompt, **kwargs):
  # Note Single quotes is important!
  return { 
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "max_tokens": 128
    }

chat_model = Databricks(allow_dangerous_deserialization=True,
                    transform_input_fn=reformat_Func,
                    endpoint_name=chat_endpoint_name)

chat_model._client.external_or_foundation = True


In [0]:
from langchain_community.chat_models import ChatDatabricks

chat_model = ChatDatabricks(target_uri = "databricks",
                              endpoint = chat_endpoint_name)

In [0]:

from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

#Create the prompt template and include the context and question
Template = """You are an engineer who manages operational equipment. If the questions are not related to the topics, you can decline to answer. If you do not know the answer, just respond with I don't know, do not try to make up an answer. Keep the answers concise. Summarise the list of resolution for an engineering issue of {question} in 100 words or less using the context below:
Context = {context}
Question = {question}"""


prompt = PromptTemplate(template = Template, input_variables = ['context','question'])


In [0]:
#Assemble the complete RAG chain
chain = RetrievalQA.from_chain_type(
  llm = chat_model,
  chain_type = "stuff",
  retriever = get_retriever(),
  chain_type_kwargs = {"prompt":prompt}

)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


In [0]:
query = "Issues related to pump failure?"

question = {"query":query}
response = chain.invoke(question)

print(response['result'])

Here is a summary of the resolution for issues related to pump failure in 100 words or less:

The main issue is pump failure, specifically Pump No.1 tripping out and blowing the main switch. The solution is to repair or replace the pumps. The repair plan is to be replanned after pumps arrived yesterday. Additionally, the supernatant pump cable needs to be removed from the junction box to prevent the well from filling up again.


###7) Save the model to UC

In [0]:
import mlflow, langchain
mlflow.set_registry_uri("databricks-uc")
from mlflow.models import infer_signature

model_name = "engineering_small_notes"
model_name_3_level = f"{catalog}.{schema}.{model_name}"

with mlflow.start_run(run_name = "engineering_small_notes") as run:
  logged_model = mlflow.langchain.log_model(
    chain,
    loader_fn = get_retriever,
    artifact_path="chain",
    input_example = query,
    signature = infer_signature(query,response),
    registered_model_name = model_name,
    pip_requirements=[
          "langchain==0.2.0",
          "databricks-vectorsearch",
          "mlflow",
          "langchain_community"
    ]
  )


/local_disk0/.ephemeral_nfs/envs/pythonEnv-59698e7e-ad03-4f66-bd6b-63167a9b4c85/lib/python3.10/site-packages/mlflow/langchain/utils.py:666: UserWarning: Using custom serializer to pickle pydantic.fields.ModelField classes, this might miss some fields and validators. To avoid this, please upgrade pydantic to v2 using `pip install pydantic -U` with langchain 0.0.267 and above.
  warnings.warn(
/databricks/python/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Registered model 'engineering_small_notes' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/11 [00:00<?, ?it/s]

Created version '19' of model 'beatrice_liew.aggreko.engineering_small_notes'.


### 8) Deploy the model as a serving endpoint
This will allow this endpoint to be queried via an API

In [0]:
from mlflow import MlflowClient
client = MlflowClient()
#Check for the latest version on UC to deploy
def get_latest_model_version(model_name_3_level):
    mlflow_client = MlflowClient()
    latest_version = 1
    for mv in mlflow_client.search_model_versions(f"name='{model_name_3_level}'"):
        version_int = int(mv.version)
        if version_int > latest_version:
            latest_version = version_int
    return latest_version

print(get_latest_model_version(model_name_3_level))

19


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedEntityInput
from datetime import timedelta
from mlflow import MlflowClient

client = MlflowClient()

w = WorkspaceClient()
serving_endpoint_name = 'engineering_notes_small'

endpoint_config = EndpointCoreConfigInput(
    served_entities =[
      ServedEntityInput(
      entity_name=f"{catalog}.{schema}.{model_name}",
      entity_version = get_latest_model_version(model_name_3_level),
      workload_size = "Small",
      scale_to_zero_enabled = True,
      workload_type = "CPU",
      environment_vars = {
        "DATABRICKS_TOKEN":"{{secrets/hackathon_beatrice/hackathon_token}}"
      }
      )
    ]
  )

existing_endpoint = next(
    (e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None
)

if existing_endpoint == None:
  w.serving_endpoints.create_and_wait(
    name = serving_endpoint_name,
    config = endpoint_config,
    timeout= timedelta(hours = 0, minutes = 40)
  )
  print("endpoint is being created")
else:
  w.serving_endpoints.update_config_and_wait(
    served_entities=endpoint_config.served_entities, 
    name=serving_endpoint_name,
    timeout= timedelta(hours = 0, minutes = 40)
    )

### 9) Query the endpoint

In [0]:
import mlflow.deployments
deploy_client = mlflow.deployments.get_deploy_client("databricks")

question = "What are the typical resolution to pump failures"
response = deploy_client.predict(endpoint="engineering_notes_small", inputs={
  "dataframe_split": {
    "data": [
      [
        question
      ]
    ]
  }
})

In [0]:
print(response['predictions'])

['Typical resolutions to pump failures in this context:\n\n* Repair or replace faulty pump hoses\n* Inspect and replace faulty inlet flow meters\n* Inspect and repair or replace pumps (Pump No.1 tripping out and blowing the main switch)\n* Plan and execute a job to repair or replace pumps (Pump No.1 and other pump)']
